In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
DBNAME = "opportunity_youth"
conn = psycopg2.connect(dbname=DBNAME)

import clean_data_functions as cdf

In [28]:
def get_oy_db():
    #fetching oportunity youth in south king county
    skc_OY_df = pd.read_sql('''
        SELECT *
        FROM pums_2017
        WHERE puma SIMILAR TO '1161(0|1|2|3|4|5)'
        AND agep >= 16
        AND agep <= 24
        AND sch = '1'
        AND esr SIMILAR TO '%(3|6)%'
        ''', conn)
    
    return skc_OY_df

def get_all_youth_db():
    #fetching all residents from south king county within the OY age group

    skc_allRes_df = pd.read_sql('''
        SELECT *
        FROM pums_2017
        WHERE puma SIMILAR TO '1161(0|1|2|3|4|5)'
        AND agep >= 16
        AND agep <= 24
        ''', conn)
    
    return skc_allRes_df

<b> 
    - break down OY by race
    - break down OY by pumsID
    - break down total youth by race
    - break down total youth by pumsID

In [21]:
def get_skc_oy_race():
    '''
    returns a dictionary with race names as keys and their coresponding pop_count as values for skc opportunity youth
    '''
    
    
    race_dict = {'1': 'White', '2': 'Black/ African American',
                 '3': 'American Indian or Alaska Native', '4': 'American Indian or Alaska Native',
                 '5': 'American Indian or Alaska Native', '6': 'Asian', '7': 'Native Hawaian/ Paciffic Islander',
                 '8': 'Other', '9': 'Two or More Races'}
    race_breakdown = skc_OY_df.groupby(by='rac1p').sum()['pwgtp']
    out_dict = {}
    for index in race_breakdown.index:
        if index in ['4', '5']:
            out_dict[race_dict[index]] += race_breakdown[index]
        else:
            out_dict[race_dict[index]] = race_breakdown[index]
    return out_dict

get_skc_oy_race()

{'White': 5269.0,
 'Black/ African American': 1315.0,
 'American Indian or Alaska Native': 347.0,
 'Asian': 1189.0,
 'Native Hawaian/ Paciffic Islander': 373.0,
 'Other': 965.0,
 'Two or More Races': 1156.0}

In [27]:
def get_skc_all_youth_race():
    '''
    returns a dictionary with race names as keys and their coresponding pop_count as values for all skc youth
    '''
    
    
    race_dict = {'1': 'White', '2': 'Black/ African American',
                 '3': 'American Indian or Alaska Native', '4': 'American Indian or Alaska Native',
                 '5': 'American Indian or Alaska Native', '6': 'Asian', '7': 'Native Hawaian/ Paciffic Islander',
                 '8': 'Other', '9': 'Two or More Races'}
    race_breakdown = skc_allRes_df.groupby(by='rac1p').sum()['pwgtp']
    out_dict = {}
    for index in race_breakdown.index:
        if index in ['4', '5']:
            out_dict[race_dict[index]] += race_breakdown[index]
        else:
            out_dict[race_dict[index]] = race_breakdown[index]
    return out_dict

get_skc_all_youth_race()

{'White': 45663.0,
 'Black/ African American': 8920.0,
 'American Indian or Alaska Native': 961.0,
 'Asian': 13328.0,
 'Native Hawaian/ Paciffic Islander': 1877.0,
 'Other': 7298.0,
 'Two or More Races': 7836.0}

In [45]:
def get_pums_youth_count():
    '''
    returns a dictionary with puma ID number as keys and their corresponding total youth count as values
    '''
    skc_all_youth_df = get_all_youth_db()
    
    puma_breakdown = skc_all_youth_df.groupby(by='puma').sum()['pwgtp']
    puma_names = pd.read_sql('''
            SELECT *
            FROM puma_names_2010
            WHERE puma SIMILAR TO '1161(0|1|2|3|4|5)'
            ''', conn)
    return pd.concat([puma_names.set_index('puma')[['puma_name']], puma_breakdown], axis = 1)
    
get_pums_youth_count()

,puma_name,pwgtp
11610,"King County (Central)--Renton City, Fairwood, ...",14043.0
11613,King County (Southwest Central)--Kent City ...,16421.0
11614,King County (Southwest)--Auburn City & Lakelan...,14316.0
11615,"King County (Southeast)--Maple Valley, Covingt...",11935.0
11611,"King County (West Central)--Burien, SeaTac, Tu...",13924.0
11612,"King County (Far Southwest)--Federal Way, Des ...",15244.0


In [37]:
def get_pums_oy_count():
    '''
    returns a dictionary with puma ID number as keys and their corresponding opportunity youth count as values
    '''
    skc_oy_df = get_oy_db()
    
    puma_breakdown = skc_oy_df.groupby(by='puma').sum()['pwgtp']
    return puma_breakdown

get_pums_oy_count()

puma
11610    1853.0
11611    2038.0
11612    1977.0
11613    2006.0
11614    1530.0
11615    1210.0
Name: pwgtp, dtype: float64